## 這是Client端

2018/03/11
RemChat 開發正式開始


In [ ]:
from socket import socket, AF_INET, SOCK_STREAM
import threading
import time
from tkinter import *
import tkinter as tk
import os
from PIL import ImageTk as itk, Image
import json, types,string
import wx
import wx.xrc

# client端的socket物件
class SocketClient(socket):
#     聊天室視窗
    window=''
#     儲存server的socket
    serverip='localhost'
    serverport='5566'
    serversocket=''
#     接收server端資料的thread    
    recv_thread=''
#儲存聊天室的訊息
    chat_text_list=[]
    
#     =====================
#     HERE IS SERVER INIT
#     =====================    
    def __init__(self,window_):
        self.serversocket= socket(AF_INET, SOCK_STREAM)
        self.window = window_

#     負責接收server端資料
#     會用threading呼叫執行持續監聽的動作
#     目前收到資料時只將資料顯示出來
    def recieve(self):
        while 1:
            data = self.serversocket.recv(8192)
            jdata = json.loads(data)
            print(jdata['cmd'])
            recv_cmd = jdata['cmd']
            
            if recv_cmd == 'disconnect':
                break
            elif recv_cmd == 'say':            
                self.chat_text_list.append([0,jdata['name'],jdata['data']])
                update_chat_text()
        self.disconnect(client)

#     程式剛開始運行時會執行run()
#     進行與SERVER連接的動作
    def run(self,ip,port):
        self.serverip = ip
        print(1)
        self.serverport = port
        print(2)
        try:
            conn_string="與伺服器(%s:%s)進行連接" % (self.serverip,self.serverport,)
            showText(self,"Rem",conn_string)
            self.serversocket.connect((self.serverip, int(self.serverport)))
            self.onconnect()
            threading.Thread(target=self.recieve).start()
        except Exception as ex:
            print (ex)
            self.onconnectfailure()
        finally:
            pass

#     sendString()負責傳送一段句子到server
    def sendString(self,string):
        self.serversocket.send(str.encode(string)) 

#     sendString()負責從server取得當前時間
    def getTime(self):
        msg = {'cmd':'what time','name':'', 'data':''}
        jmsg = json.dumps(msg)
        self.serversocket.send(str.encode(jmsg))

#     disconnect()負責從server取得當前時間
    def disconnect(self):
        self.onclose(client)
        #Closing connection with client
        self.serversocket.close()
        #Closing thread
        if(recv_thread!=None):
            recv_thread.exit()
            
    # ===================
    # SOME EVENT FUNCTION
    # ===================
    def onconnectfailure(self):
        showText(self,"Rem","目標電腦拒絕連線")
    def onconnect(self):
        showText(self,"Rem","成功與伺服器端連接")
        
    def onmessage(self, message):
        print ("Got message ",message)
        
    def onclose(self, client):
        print ("Disconnected")
# ==========================
# 以下為一些函式
# ==========================
#   更新聊天室的訊息    
chating_room=''
chat_text=''
current_text_index=-1
def update_chat_text():
    global current_text_index,chating_room,chat_text
    while current_text_index<len(chating_room.chat_text_list)-1:
        current_text_index+=1
        the_string = chating_room.chat_text_list[current_text_index][1]+"說:"+chating_room.chat_text_list[current_text_index][2]+'\n'
        chat_text.configure(state="normal")
        chat_text.insert(INSERT, the_string)
        chat_text.configure(state="disabled")
    chat_text.see("end")
        
#   本地端新增文字到聊天室的訊息上
def showText(chating_room,speaker,text):
    chating_room.chat_text_list.append(['0',speaker,text])
    update_chat_text()
#   取得本地端檔案路徑

def getFilePath(filename):
    script_dir = os.getcwd() #<-- absolute dir the script is in
    return os.path.join(script_dir, filename)


# ==========================
# 以下為主程式碼
# ==========================
def main():
    global chating_room,chat_text
    current_text_index=-1
    
#     產生程式視窗並初始化
    window = tk.Tk()
    window.title('RemChat')
    window.geometry('1200x600')
    print(window.winfo_width())
    
#     產生一個聊天室
    chating_room = SocketClient(window)

#    帳號設定
    username="Pohong"
    def setShape(widget,dict_,anchor_):
        widget.place(x=dict_['x'], y=dict_['y'], width=dict_['width'], height=dict_['height'], anchor=anchor_)

# ==========================
# 以下程式碼負責聊天室文字顯示
# ==========================
#     一些參數設定
    TEXT_SIZE=15

    CHAT_TEXTBOX_SHAPE={'x':10,'y':10,'width':440,'height':300}
    CHAT_TEXTBOX_MAX_LINE=5
    CHAT_TEXTB0X_SCROLLBAR_SHAPE={'x':460,'y':10,'width':30,'height':300}
    
    INPUTBOX_SHAPE={'x':10,'y':320,'width':320,'height':100}
    INPUTBOX_SCROLLBAR_SHAPE={'x':330,'y':320,'width':30,'height':100}
    
    SEND_MSG_BTN_SHAPE={'x':380,'y':320,'width':100,'height':100}
#     聊天室訊息視窗中
    chat_text_scrollbar = Scrollbar(window) 

    chat_text = tk.Text(window,yscrollcommand=chat_text_scrollbar.set)
    chat_text.config(font=("Courier", TEXT_SIZE))
    chat_text.place(x=CHAT_TEXTBOX_SHAPE['x'], y=CHAT_TEXTBOX_SHAPE['y'], width=CHAT_TEXTBOX_SHAPE['width'], height=CHAT_TEXTBOX_SHAPE['height'], anchor='nw')
      
    chat_text_scrollbar.config(command=chat_text.yview)  
    chat_text_scrollbar.place(x=CHAT_TEXTB0X_SCROLLBAR_SHAPE['x'], y=CHAT_TEXTB0X_SCROLLBAR_SHAPE['y'], width=CHAT_TEXTB0X_SCROLLBAR_SHAPE['width'],height=CHAT_TEXTB0X_SCROLLBAR_SHAPE['height'], anchor='nw')

#     文字輸入欄位視窗
    inputbox_scrollbar = Scrollbar(window)  
    
    inputbox = tk.Text(window,  yscrollcommand=inputbox_scrollbar.set)
    inputbox.config(font=("Courier", TEXT_SIZE))
    inputbox.place(x=INPUTBOX_SHAPE['x'], y=INPUTBOX_SHAPE['y'], width=INPUTBOX_SHAPE['width'], height=INPUTBOX_SHAPE['height'], anchor='nw')
    
    inputbox_scrollbar.config(command=inputbox.yview) 
    inputbox_scrollbar.place(x=INPUTBOX_SCROLLBAR_SHAPE['x'], y=INPUTBOX_SCROLLBAR_SHAPE['y'], width=INPUTBOX_SCROLLBAR_SHAPE['width'],height=INPUTBOX_SCROLLBAR_SHAPE['height'], anchor='nw')



     
#   送出文字到SERVER
    def sendInput():
        try:
            msg = {'cmd':'say','name':username, 'data':inputbox.get("1.0",END)[:-1]}
            jmsg = json.dumps(msg)
            chating_room.sendString(jmsg)
            inputbox.delete('1.0', END)
        except Exception as ex:
            print (ex)
            showText(chating_room,"Rem","無法送出文字到SERVER QQ")
        finally:
            pass

#   視窗中的傳送訊息按鈕
    send_msg_btn = tk.Button(window, text='送出' ,command =sendInput)
    send_msg_btn.place(x=SEND_MSG_BTN_SHAPE['x'], y=SEND_MSG_BTN_SHAPE['y'], width=SEND_MSG_BTN_SHAPE['width'],height=SEND_MSG_BTN_SHAPE['height'], anchor='nw')

# ================================
# 以下程式碼負責與伺服器連接的相關設定
# =================================
#     一些參數設定
    SERVER_CONN_BTN_SHAPE={'x':410,'y':430,'width':80,'height':70}
    SERVER_IP_ENTRY_SHAPE={'x':210,'y':430,'width':180,'height':30}
    SERVER_PORT_ENTRY_SHAPE={'x':210,'y':470,'width':180,'height':30}
    TXT_SERVER_IP_ENTRY_SHAPE={'x':10,'y':430,'width':180,'height':30}
    TXT_SERVER_PORT_ENTRY_SHAPE={'x':10,'y':470,'width':180,'height':30} 
    
    USERNAME_SHAPE={'x':210,'y':510,'width':180,'height':30}
    TXT_USERNAME_SHAPE={'x':10,'y':510,'width':180,'height':30}     
#   SERVER的IP設定
    txt_ip = StringVar(window, value='Server IP:')
    txt_server_ip_entry= Entry(window,textvariable=txt_ip)
    txt_server_ip_entry.config(font=("Courier", TEXT_SIZE),state="disabled")
    txt_server_ip_entry.place(x=TXT_SERVER_IP_ENTRY_SHAPE['x'], y=TXT_SERVER_IP_ENTRY_SHAPE['y'], width=TXT_SERVER_IP_ENTRY_SHAPE['width'], height=TXT_SERVER_IP_ENTRY_SHAPE['height'], anchor='nw')
    
    txt_ip = StringVar(window, value='localhost')
    server_ip_entry = Entry(window,textvariable=txt_ip)
    server_ip_entry.place(x=SERVER_IP_ENTRY_SHAPE['x'], y=SERVER_IP_ENTRY_SHAPE['y'], width=SERVER_IP_ENTRY_SHAPE['width'], height=SERVER_IP_ENTRY_SHAPE['height'], anchor='nw')

#   SERVER的Port設定 
    txt_port = StringVar(window, value='Server Port:')
    txt_server_port_entry = Entry(window, textvariable=txt_port)
    txt_server_port_entry.config(font=("Courier", TEXT_SIZE),state="disabled")
    txt_server_port_entry.place(x=TXT_SERVER_PORT_ENTRY_SHAPE['x'], y=TXT_SERVER_PORT_ENTRY_SHAPE['y'], width=TXT_SERVER_PORT_ENTRY_SHAPE['width'], height=TXT_SERVER_PORT_ENTRY_SHAPE['height'], anchor='nw')
    
    txt_port = StringVar(window, value='5566')
    server_port_entry = Entry(window,textvariable=txt_port)
    setShape(server_port_entry,SERVER_PORT_ENTRY_SHAPE,'nw')
    
#   使用者的名字設定
    txt_user = StringVar(window, value='User Name:')
    txt_username_entry = Entry(window, textvariable=txt_user)
    txt_username_entry.config(font=("Courier", TEXT_SIZE),state="disabled")
    setShape(txt_username_entry,TXT_USERNAME_SHAPE,'nw')
    
    txt_user = StringVar(window, value='Pohong')
    username_entry = Entry(window,textvariable=txt_user)
    setShape(username_entry,USERNAME_SHAPE,'nw')
    
#   與SERVER連接的按鈕
    server_conn_btn = tk.Button(window, text='連線' ,command = lambda: chating_room.run(server_ip_entry.get(),server_port_entry.get()))
    setShape(server_conn_btn,SERVER_CONN_BTN_SHAPE,'nw')
    
# ================================
# 以下程式碼負責程式特殊功能...
# =================================
#     一些參數設定
    GETTIME_BTN_SHAPE={'x':10,'y':560,'width':100,'height':30}
#   取得時間
    gettime_btn = tk.Button(window, text='現在幾點?' ,command = lambda: chating_room.getTime())
    setShape(gettime_btn,GETTIME_BTN_SHAPE,'nw')

    
#     showText(self,"Rem","客戶端開始運行")
    showText(chating_room,"Rem","歡迎使用聊天室~")
    showText(chating_room,"Rem","請輸入伺服器ID和PORT進行連線~")
    window.mainloop()
    
if __name__ == "__main__":
    main()




1
1
2
saysay

